In [ ]:
import pandas as pd
import os
import numpy as np
from scipy.stats import norm

In [ ]:
def bs_price(S, K, sigma, r, T, year_rate = 252, option_type = "call"):
     '''
     bs 期权定价 输出为期权价格

     :param S: 标定价格
     :param K: 行权价
     :param sigma: 波动率
     :param r: 无风险利率
     :param T: 年化剩余期限
     :param year_rate: 年化系数
     :param option_type: "call"
     :return:
     ''''''
     '''

     T = T/year_rate
     d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
     d2 = d1 - sigma*np.sqrt(T)
     if option_type == 'call':
            # return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
              return np.maximum(S - K, 0) if T==0 else S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
     elif option_type == 'put':
              # return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
              return np.maximum(K-S,0) if T==0 else K*np.exp(-r*T)*norm.cdf(-1*d2) - S*norm.cdf(-1*d1)
     else:
              print("期权类型错误")
              return -1


In [ ]:
def bs_vega(S,K,sigma,r,T ,year_rate = 252):
          '''
          计算vega
          '''
          T = T/year_rate
          d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(np.abs(T)) )
          return S*norm.pdf(d1)*np.sqrt(np.abs(T))
          # return S * np.sqrt(abs(T)) * np.exp(-pow(d1,2)/2) / np.sqrt(2*np.pi)

In [ ]:
def iv_bs_bisection(S, K, r, T, price, option_type, iv_uplimit = 1.0, iv_downlimit = 0.000001, precision = 1, year_rate = 252, max_iterations = 200):
          '''二分法求iv'''
          left_iv, right_iv = iv_downlimit, iv_uplimit
          # left_price = BS_price(S,K,left_iv, r, T,     year_rate = year_rate, option_type = option_type)
          # right_price= BS_price(S,K,right_iv, r, T,     year_rate = year_rate, option_type = option_type)
          mid_iv = (left_iv + right_iv)/2
          mid_price = bs_price(S,K,mid_iv, r, T,     year_rate = year_rate, option_type = option_type)
          # print(f"first, the mid price is {mid_price}, the price is {price}")
          cnt = 0
          while abs(price - mid_price) >= 0.1**precision and cnt < max_iterations:
                    if mid_price < price :
                              left_iv = mid_iv
                    else:
                              right_iv = mid_iv
                    mid_iv = (left_iv + right_iv)/2
                    # if mid_iv <= iv_uplimit or mid_iv >= iv_downlimit:
                    #           break
                    # print(f"cnt = {cnt}, the left is{left_iv}, the right is{right_iv}, the

In [ ]:
def find_vol_newton( S, K, r, T, target_value, option_type, start_sigma = 0.5, precision = 3, year_rate = 252, max_iterations = 200):
          # 迭代法求iv
          sigma = start_sigma
          for i in range(0, max_iterations):
                    bs_price_ = bs_price(S,K,sigma,r,T, year_rate = year_rate, option_type = option_type)
                    vega = bs_vega(S, K, T, r, sigma, year_rate = year_rate)*100
                    diff = target_value - bs_price_     # our root
                    if (abs(diff) < 0.1**precision):
                              return sigma
                    sigma = sigma + diff/(vega) # f(x) / f'(x)
          return sigma # value wasn't found, return best guess so far